![Logo dell'Università XYZ](img/logo.jpg)

# Advanced Financial Modeling Course

## Introduction

This notebook is part of the teaching material of the Advanced Financial Modeling course! In this series of Jupyter notebooks, we will cover various topics related to financial modeling, including fundamental concepts, practical applications, and hands-on exercises.

### Notebooks Overview

1. [Notebook 1: Curve Building](notebooks/notebook1.ipynb)
2. [Notebook 2: AAD Greeks](notebooks/notebook2.ipynb)
3. [Notebook 3: Swap AAD Sensitivities](notebooks/notebook3.ipynb)
3. [Notebook 4: Simulation of Extended Short-Rate Models](notebooks/notebook3.ipynb)
4. [Notebook 5: Model Calibration](notebooks/notebook4.ipynb)
5. [Notebook 6: SABR stochastic volatility](notebooks/notebook5.ipynb)
5. [Notebook 7: xVA--CCR simulation](notebooks/notebook5.ipynb)


## Notebook 2: AAD Greeks

This notebook provides a brief overview of Automatic Adjoint Differentiation (AAD) and its relevance in financial modeling. AAD is a technique used to efficiently compute sensitivities (e.g., Greeks) of financial instruments with respect to various parameters. It's particularly useful in derivatives pricing and risk management, as it allows for accurate and fast computation of sensitivities, such as delta, gamma, theta, etc.

### Topics Covered

- Understanding the importance and limitations of automatic differentiation in finance.
- Using tensorflow for option-pricing.
- Difference with standard methods.





## References
- Savine, Antoine. Modern computational finance: AAD and parallel simulations. John Wiley & Sons, 2018.

In [1]:
import matplotlib.pyplot as plt
import QuantLib as ql 

from abc import ABC, abstractmethod
import tensorflow as tf
import tensorflow_probability as tfp

## Option Parameters

In [2]:
class EuropeanOptionInstrument(ABC):
    def __init__(self, strike_price, maturity, option_type):
        self.strike_price = tf.Variable(strike_price, dtype=tf.float32)
        self.maturity = tf.Variable(maturity, dtype=tf.float32)
        self.option_type = option_type  # "Call" or "Put"

    @abstractmethod
    def calculate_payoff(self, market_price):
        pass

    @abstractmethod
    def calculate_intrinsic_value(self, market_price):
        pass

    @abstractmethod
    def calculate_time_value(self, market_price):
        pass

    def display_information(self):
        print(f"Strike Price: {self.strike_price}")
        print(f"Maturity: {self.maturity} years")
        print(f"Option Type: {self.option_type}")

    @property
    @abstractmethod
    def is_exercisable(self):
        pass

class EuropeanCallOption(EuropeanOptionInstrument):
    def calculate_payoff(self, market_price):
        return max(market_price - self.strike_price, 0)

    def calculate_intrinsic_value(self, market_price):
        return max(market_price - self.strike_price, 0)

    def calculate_time_value(self, market_price):
        intrinsic_value = self.calculate_intrinsic_value(market_price)
        return max(0, market_price - intrinsic_value)

    @property
    def is_exercisable(self):
        return True if self.maturity > 0 else False

In [3]:
# Sample data
spot_price = 100.0
strike_price = 100.0
maturity = 1.0
option_type = "Call"
risk_free_rate = 0.03
volatility = 0.2

## Closed functions

In [5]:
class BlackScholesPricing:
    
    def __init__(self, option: EuropeanOptionInstrument, spot_price, risk_free_rate, volatility):
        self.option = option
        self.spot_price = tf.Variable(spot_price, dtype=tf.float32)
        self.risk_free_rate = tf.Variable(risk_free_rate, dtype=tf.float32)
        self.volatility = tf.Variable(volatility, dtype=tf.float32)

    def calculate_option_price(self):
        d1 = (tf.math.log(self.spot_price / self.option.strike_price) + (
            self.risk_free_rate + (self.volatility ** 2) / 2) * self.option.maturity) / (self.volatility * tf.sqrt(self.option.maturity))
        d2 = d1 - self.volatility * tf.sqrt(self.option.maturity)
        dist = tfp.distributions.Normal(0,1)
        call_option_price = self.spot_price * dist.cdf(d1) - self.option.strike_price * tf.exp(-self.risk_free_rate * self.option.maturity) * dist.cdf(d2)

        return call_option_price if self.option.option_type == "Call" else call_option_price - (self.spot_price - self.option.strike_price)*tf.exp(-self.risk_free_rate * self.option.maturity)

    def calculate_aad(self):
        with tf.GradientTape() as tape:
            tape.watch(self.spot_price)
            tape.watch(self.risk_free_rate)
            tape.watch(self.volatility)

            option_price = self.calculate_option_price()

        aad_greeks = tape.gradient(option_price, [self.spot_price, self.volatility, self.risk_free_rate ])
        #vega = tape.gradient(option_price, self.volatility)

        return option_price, aad_greeks
    
    def calculate_aad_xla(self):
        # Apply XLA compilation to the gradient calculation
        @tf.function(jit_compile=True)
        def compute_gradients():
            with tf.GradientTape() as tape:
                tape.watch(self.spot_price)
                tape.watch(self.risk_free_rate)
                tape.watch(self.volatility)

                option_price = self.calculate_option_price()

            #delta = tape.gradient(option_price, self.spot_price)
            aad_greeks = tape.gradient(option_price, [self.spot_price, self.volatility, self.risk_free_rate ])
            return option_price, aad_greeks

        return compute_gradients()    

## Montecarlo simulation

In [6]:
class EuropeanKernel:
    
    def __init__(self, option, spot_price, mu, sigma, z):
        self.option = option
        self.spot_price = tf.Variable(spot_price, dtype=tf.float32)
        self.mu = tf.Variable(mu, dtype=tf.float32)
        self.sigma = tf.Variable(sigma, dtype=tf.float32)
        self.z = z


    def calculate_option_price(self):
        dt = self.option.maturity / self.z.shape[1]
        dt_sqrt = tf.math.sqrt(dt)
        diffusion = self.sigma * dt_sqrt
        drift = (self.mu - (self.sigma ** 2) / 2)
        gbm = tf.math.exp(drift * dt + diffusion * self.z)
        s_t = self.spot_price * tf.math.cumprod(gbm, axis=1)

        payoff = tf.math.maximum(s_t[:, -1] - self.option.strike_price, 0)
        return tf.exp(-self.mu * self.option.maturity) * tf.reduce_mean(payoff)
    
    def calculate_option_price_xla(self):
        @tf.function(jit_compile = True)
        def compute_price():
            dt = self.option.maturity / self.z.shape[1]
            dt_sqrt = tf.math.sqrt(dt)
            diffusion = self.sigma * dt_sqrt
            drift = (self.mu - (self.sigma ** 2) / 2)
            gbm = tf.math.exp(drift * dt + diffusion * self.z)
            s_t = self.spot_price * tf.math.cumprod(gbm, axis=1)

            payoff = tf.math.maximum(s_t[:, -1] - self.option.strike_price, 0)
            return tf.exp(-self.mu * self.option.maturity) * tf.reduce_mean(payoff)
        return compute_price()


    def calculate_aad(self):
        with tf.GradientTape() as tape:
            option_price = self.calculate_option_price()

        aad_greeks = tape.gradient(option_price, [self.spot_price, self.sigma, self.mu])

        return option_price, aad_greeks
    

    def calculate_aad_xla(self):
        @tf.function(jit_compile=True)
        def compute_gradients():
            with tf.GradientTape() as tape:
                option_price = self.calculate_option_price()

            aad_greeks = tape.gradient(option_price, [self.spot_price, self.sigma, self.mu])

            return option_price, aad_greeks

        return compute_gradients()

## AAD with closed form pricing

In [7]:
# Create an instrument-object for the European Call Option 
call_option = EuropeanCallOption(strike_price, maturity, option_type)
# Create an engine-object for the Black-Scholes model
bs_engine = BlackScholesPricing(call_option, spot_price, risk_free_rate, volatility)
# Calculate the call option price
call_option_price = bs_engine.calculate_option_price()
print(f"Call Option Price: {call_option_price.numpy()}")
print("*"*30)
# Calculate Delta and Vega using automatic differentiation
price, sensy = bs_engine.calculate_aad()
print("Black AAD: ")
print("Price: ", price.numpy())
print(f"Delta: {sensy[0].numpy()}")
print(f"Vega: {sensy[1].numpy()}")
print(f"Rho: {sensy[2].numpy()}")
print("*"*30)


Call Option Price: 9.413406372070312
******************************
Black AAD: 
Price:  9.413406
Delta: 0.5987063646316528
Vega: 38.666812896728516
Rho: 50.45722961425781
******************************


## AAD with Montecarlo simulation

In [8]:
n_path = 20000
timesteps = 100
z = tf.random.normal((n_path, timesteps), seed=12)

mc_engine = EuropeanKernel(call_option, spot_price, risk_free_rate, volatility, z)

In [9]:
call_option_price = mc_engine.calculate_option_price()
print(f"Call Option MC Price: {call_option_price.numpy()}")
print("*"*30)
# Calculate Delta and Vega using automatic differentiation
price, sensy = mc_engine.calculate_aad()
print("Black AAD MC: ")
print("Price: ", price.numpy())
print(f"Delta: {sensy[0].numpy()}")
print(f"Vega: {sensy[1].numpy()}")
print(f"Rho: {sensy[2].numpy()}")
print("*"*30)

Call Option MC Price: 9.448568344116211
******************************
Black AAD MC: 
Price:  9.448568
Delta: 0.5953810811042786
Vega: 38.992950439453125
Rho: 50.08954620361328
******************************


## Analytic form for AD